In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re
import time
import datetime
import requests
import numpy as np

In [ ]:
ins_2024 = pd.read_csv('/content/drive/MyDrive/Restaurant inspections/2024_inspection.csv')
ins_2023 = pd.read_csv('/content/drive/MyDrive/Restaurant inspections/2023_inspection.csv')

In [ ]:
df = pd.concat([ins_2024, ins_2023])

In [ ]:
df.head()

,Unnamed: 0,Business Name,Address,City,Permit ID,Type,Class,Inspection Date,Inspection Type,Grade,Priority Violations,Cutting Edge,Inspection Details
0,0,Phoenix Silvercrest,613 N 4th Ave,Phoenix,FD-00003,E & D 10+ Seating,Class 4,01/03/2024,Routine Inspection,Not Participating,1,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...
1,1,Alexis Grill,3550 N Central Ave Suite 120,Phoenix,FD-00464,E & D 10+ Seating,Class 4,01/05/2024,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...
2,2,Schreiner's Fine Sausage,3601 N 7th St,Phoenix,FD-00501,Meat Market,Class 4,01/05/2024,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...
3,3,Circle K,7602 E Mckellips Rd,Scottsdale,FD-00640,E & D 0-9 Seating,Class 2,01/05/2024,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...
4,4,Circle K,7602 E Mckellips Rd,Scottsdale,FD-00641,Retail Food Establishment,Class 2,01/05/2024,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...


In [ ]:
df['Address'] = df['Address'].str.strip()
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])
df = df[df['Inspection Date'] > '2023-04-21']
df['Year'] = df['Inspection Date'].dt.year

<ipython-input-6-5ec2f6b8238c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Year'] = df4['Inspection Date'].dt.year


In [ ]:
df[df['Year'] == 2023].sort_values(by='Inspection Date', ascending=True).head()

,Unnamed: 0,Business Name,Address,City,Permit ID,Type,Class,Inspection Date,Inspection Type,Grade,Priority Violations,Cutting Edge,Inspection Details,Year
18438,18438,Fantasy Funnel Cake,Special Events,NaN,MF-14-0514,Mobile Food Type III,Class 4,2023-04-22,Routine Inspection,A,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2023
17449,17449,Phoenix Park N Swap - Pizza & Ice Cream,3801 E Washington St,Phoenix,FD-00853,E & D 0-9 Seating,Class 3,2023-04-22,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2023
18474,18474,Yallatizers AZ,,NaN,MF-21-0104,Mobile Food Type I,Class 2,2023-04-22,Immediate Suspension,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2023
18469,18469,Simple Soda Co,@6@2,NaN,MF-20-0496,Annual Event Food Est,Class 2,2023-04-22,Routine Inspection,A,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2023
18520,18520,Love Acai Bowls,Special Events @6@2,Phoenix,MF-22-0304,Mobile Food Type III,Class 4,2023-04-22,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2023


In [ ]:
# Path to save the scraped data
file_path = 'scraped_data.csv'

# Function to save progress
def save_progress(df, path):
    df.to_csv(path, index=False)

# Load existing data if the file exists
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df = pd.DataFrame(columns=['Inspection Details', 'Violation_Text'])

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    url = row['Inspection Details']
    if pd.isnull(url):  # If URL is NaN, set "This row has no violation"
        df.at[index, 'Violation_Text'] = "This row has no violation"
        continue
    print(f"Scraping row {index + 1}: {url}")
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = bs(response.text, 'html.parser')
            violation_element = soup.find(class_="col-5 CellJustify")
            if violation_element:  # Check if element was found
                violation_text = violation_element.text.strip()
                df.at[index, 'Violation_Text'] = violation_text
            else:
                df.at[index, 'Violation_Text'] = "No violation text found"
        else:
            df.at[index, 'Violation_Text'] = "Error: Unable to fetch data from URL"
    except Exception as e:
        print(f"Error: {e}")
        df.at[index, 'Violation_Text'] = "Error: Unable to fetch data from URL"

    time.sleep(0.5)  # Sleep for 0.5 seconds to avoid overwhelming the server

    # Save progress every 100 rows
    if index % 100 == 0:
        save_progress(df, file_path)
        print(f"Progress saved at row {index}")

# Save final progress
save_progress(df, file_path)
print("Final progress saved")

Streaming output truncated to the last 5000 lines.
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04460&i=5652075
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04461&i=5652069
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04465&i=5651003
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04467&i=5650992
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04551&i=5648986
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04559&i=5652020
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04560&i=5652019
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04561&i=5652013
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04562&i=5652021
Scraping http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-04672&i=5649928
Scraping http://envapp.marico

KeyboardInterrupt: 

In [ ]:
df4[df4['Violation_Text'].isnull()].head()

,Unnamed: 0,Business Name,Address,City,Permit ID,Type,Class,Inspection Date,Inspection Type,Grade,Priority Violations,Cutting Edge,Inspection Details,Year,Violation_Text
12177,12177,Monte Vista Village Resort,8865 E Baseline Rd,Mesa,FD-28746,E & D 0-9 Seating,Class 3,2024-03-14,Routine Inspection,B,1,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2024,NaN
12178,12178,Grand Ave Produce,8990 W Windsor Dr,Peoria,FD-28937,Food Processor,Class 2,2024-03-12,Routine Inspection,A,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2024,NaN
12179,12179,Grand Ave Produce,8990 W Windsor Dr,Peoria,FD-28938,Food Jobber,Class 2,2024-03-12,Routine Inspection,A,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2024,NaN
12180,12180,Roliberto's Mexican Food,12555 NW Grand Ave,El Mirage,FD-29734,E & D 10+ Seating,Class 4,2024-03-14,Routine Inspection,A,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2024,NaN
12181,12181,Bambino's Sports Grill,3860 W Happy Valley Rd Suite 102,Phoenix,FD-30074,E & D 10+ Seating,Class 4,2024-03-15,Routine Inspection,Not Participating,0,NaN,http://envapp.maricopa.gov/EnvironmentalHealth...,2024,NaN
